In [8]:
import pandas as pd
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

In [9]:
from src.data.provide_data import get_model_data
from src.models.evaluate import custom_classification_report, custom_lazy_report



In [11]:
X_train, y_train = get_model_data(filename = "Train")
X_valid, y_valid = get_model_data(filename = "Valid")
X_test, y_test = get_model_data(filename = "Test")

cat = [X_train.columns.get_loc(i) for i in ["Team","Div","Opponent"]]

Train 5246
Valid 1952
Test 1952


In [12]:
features = X_train.columns

In [13]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler object
scaler = StandardScaler()

# Fit scaler on training data
scaler.fit(X_train)

# Scale training and test data
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

# Baseline Model

In [ ]:
from lazypredict.Supervised import LazyClassifier

lazy_clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = lazy_clf.fit(X_train, X_valid, y_train, y_valid)
models

In [ ]:
custom_lazy_report(X_test,y_test,lazy_clf)

In [15]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search over
params = {'learning_rate': 0.01007397282357752, 'num_leaves': 12, 'max_depth': 3, 'min_child_samples': 6}

# Create a LGBMClassifier model
clf = LGBMClassifier(**params)  # Use **params to pass the dictionary as keyword arguments

# Fit the GridSearchCV object to the training data
clf.fit(X_train, y_train)

# Print the best hyperparameters found by GridSearchCV

# Call the custom_classification_report function
custom_classification_report(X=X_test, y=y_test, model=clf)

import pickle

pickle.dump(clf, open(CONFIG.DATA_FOLDER_MODELS + 'lgb.pkl', 'wb'))

[LightGBM] [Info] Number of positive: 1966, number of negative: 3280
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001776 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7445
[LightGBM] [Info] Number of data points in the train set: 5246, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.374762 -> initscore=-0.511842
[LightGBM] [Info] Start training from score -0.511842
              precision    recall  f1-score   support

       False       0.68      0.92      0.78      1212
        True       0.69      0.30      0.41       740

    accuracy                           0.68      1952
   macro avg       0.69      0.61      0.60      1952
weighted avg       0.68      0.68      0.64      1952

AUC 0.706


In [23]:
clf.booster_.feature_importance(importance_type='split')  # 'split' or 'gain'

array([16,  0, 55, 41, 11, 11,  0, 89, 37,  2,  0,  1,  2,  0,  0, 39,  1,
       15,  0,  0,  0,  0, 34,  0,  9, 11, 31,  0,  0, 33, 97,  4,  0,  0,
        0,  0, 20, 22, 20, 31,  1,  9, 20,  5, 33])

## Gridsearch für bestes Baseline Model

In [36]:
fi = pd.DataFrame() 
fi['features'] = features
fi['importance'] = clf.booster_.feature_importance(importance_type='gain')  
fi['importance'] = fi['importance']/sum(fi['importance'])
fi.sort_values(by='importance',ascending=False).head(10)

,features,importance
7,B365_Team,0.468098
30,B365_Team_odd_pred,0.142169
36,BW_Team_odd_pred,0.077514
22,Avg_Opponent,0.047664
17,Avg_Team,0.039862
3,BW_Draw,0.036606
2,BW_Opponent,0.031168
26,Min_Draw,0.028843
15,Max_Team,0.025619
5,B365_Opponent,0.025413


,features,importance
30,B365_Team_odd_pred,0.138571
7,B365_Team,0.127143
2,BW_Opponent,0.078571
3,BW_Draw,0.058571
15,Max_Team,0.055714
8,IW_Opponent,0.052857
22,Avg_Opponent,0.048571
29,Ratio_Draw,0.047143
44,Span_Draw_last_4_games,0.047143
39,B365_Team_last_4_games,0.044286


In [ ]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search over
param_grid = {
    'learning_rate': [0.01, 0.1, 1.0],
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 4, 5]
}
params = {'learning_rate': 0.01007397282357752, 'num_leaves': 12, 'max_depth': 3, 'min_child_samples': 6}
# Create a LGBMClassifier model
clf = LGBMClassifier(params)

# Create a GridSearchCV object to perform grid search
grid_search = GridSearchCV(clf, param_grid, cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found by GridSearchCV
print("Best hyperparameters:", grid_search.best_params_)
custom_classification_report(X = X_test,y = y_test,model = grid_search)


In [ ]:
import optuna
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

# Define an objective function to optimize
def objective(trial):
    # Define the search space for hyperparameters
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
    }

    # Split your data into training and validation sets
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create a LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)

    # Specify the number of boosting rounds
    num_boost_round = 10000

    # Initialize variables for early stopping
    early_stopping_rounds = 100
    early_stopping_counter = 0
    best_logloss = float('inf')

    # Train the LightGBM model
    clf = lgb.LGBMClassifier(**params, n_estimators=num_boost_round)
    clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])

    # Make predictions on the validation set
    val_pred = clf.predict_proba(X_valid, num_iteration=clf.best_iteration_)[:, 1]
    logloss = log_loss(y_valid, val_pred)

    if logloss < best_logloss:
        best_logloss = logloss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_rounds:
            return best_logloss  # Early stopping

    return best_logloss

# Create a study object and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params

# Split your data into training and validation sets
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a LightGBM classifier with the best hyperparameters
clf = lgb.LGBMClassifier(**best_params, n_estimators=100)  # You can specify a large number of estimators
clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100)

# Make predictions on the validation set
y_pred = clf.predict(X_valid)

# Calculate accuracy on the validation set
accuracy = accuracy_score(y_valid, y_pred)
print("Validation Accuracy:", accuracy)


# Transformer

## Pretrainer

In [ ]:

import torch

from pytorch_tabnet.pretraining import TabNetPretrainer

# Define pretrainer model architecture
pretrainer = TabNetPretrainer(
optimizer_fn=torch.optim.Adam,
optimizer_params=dict(lr=2e-2),
mask_type="entmax"
)

# Train pretrainer model on training data
pretrainer.fit(
    X_train=X_train,
    eval_set=[X_valid],
    max_epochs=1000,
    patience=30,
    pretraining_ratio=0.8,
    batch_size= 64
)


## Modellfitting

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

# Define hyperparameters
n_d = 8
n_a = 8
n_steps = 3
gamma = 1
lambda_sparse = 0.001
lr = 0.0001
batch_size = 16
max_epochs = 150

# Create TabNet classifier
clf = TabNetClassifier(n_d=n_d, n_a=n_a, cat_dims=cat,n_steps=n_steps, gamma=gamma, lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=lr), mask_type='entmax', device_name='cuda' if torch.cuda.is_available() else 'cpu')
# Train TabNet classifier
clf.fit(X_train=X_train, y_train=y_train,     eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'], eval_metric=['auc','balanced_accuracy'],batch_size=batch_size, max_epochs=max_epochs, patience=0)
# from_unsupervised=pickle.load(open('tabnet.pkl', 'rb'))

import pickle
pickle.dump(clf, open('tabnet.pkl', 'wb'))

In [ ]:
import pickle
pickle.dump(clf, open('tabnet.pkl', 'wb'))


In [ ]:
pickle.load(open('tabnet.pkl', 'rb'))

## Evaluierung

In [ ]:
from matplotlib import pyplot as plt

plt.plot(clf.history['loss'], label='Validation Loss')
plt.plot(clf.history['valid_auc'], label='AUC')
plt.plot(clf.history['valid_balanced_accuracy'], label='Accuracy')

plt.legend()
plt.show()

from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
r = classification_report(y_pred=y_pred,y_true=y_test)
print(r)

## Erklärbarkeit

In [ ]:
f_i = pd.Series(clf.feature_importances_)
f_i.index = features
f_i.sort_values(ascending=False)

In [ ]:
"BW_opponent_odd_pred","B365_Opponent","Avg_Opponent","IW_Team","Span_Draw"      

In [ ]:
sum(f_i>0.01)

#  Hypertune

In [ ]:
import optuna
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, brier_score_loss

def load_dataset():
    # Load your dataset here
    # Replace this with your dataset loading code
    # X and y should be your feature and target variables
    pass

def objective(trial):
    # Define the hyperparameters to optimize
    shrink_threshold = trial.suggest_float("shrink_threshold", 0.0, 1.0)
    
    X_train, y_train = get_model_data(filename = "Train")
    X_valid, y_valid = get_model_data(filename = "Valid")
    X_test, y_test = get_model_data(filename = "Test")
    
    # Create and train the NearestCentroid classifier with the suggested parameters
    clf = NearestCentroid(shrink_threshold=shrink_threshold)
    clf.fit(X_train, y_train)
    
    # Make predictions on the validation set
    y_pred = clf.predict(X_valid)
    
    # Calculate the accuracy of the model
    score = brier_score_loss(y_valid, y_pred)
    
    return brier_score_loss

if __name__ == "__main__":
    # Load your dataset
    
    # Specify the SQLite database file for the study
    study_name = "neareast_centroid_optimization.db"
    study = optuna.create_study(direction="minimize", study_name=study_name)

    # Optimize the study
    study.optimize(objective, n_trials=100)  # You can adjust the number of trials

    # Get the best parameters and their corresponding accuracy
    best_params = study.best_params
    best_accuracy = study.best_value

    print("Best Parameters:", best_params)
    print("Best Accuracy:", best_accuracy)

    # # Save the study to the specified SQLite database file
    # study.trials_dataframe().to_sql(study_name, "sqlite:///{}".format(study_name), if_exists="replace")


In [ ]:
    clf = NearestCentroid(shrink_threshold=0.95)
    clf.fit(X_train, y_train)

In [ ]:
clf.predict(X_test)